 # Multisearch Agent RAG

###  1. Environment and API Key Setup

In [223]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['GEMINI_API_KEY'] = os.getenv("GEMINI_API_KEY")

###  2. LLM Initialization

In [224]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             temperature=0,
                             google_api_key=os.getenv("GEMINI_API_KEY"))


###  3. Tool Definitions

####  Tool 1: Wikipedia

In [225]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### Tool 2: Web Page Retriever (for LangSmith)

In [303]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

cohere_embeddings = CohereEmbeddings(model="embed-english-v3.0")
loader = WebBaseLoader("https://nitsri.ac.in/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, cohere_embeddings)

In [308]:
retriever = vectordb.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about NIT srinagar .  For any questions about National Institute of Technology Srinagar use this tool."
)

#### Tool 3: Arxiv for scientific papers

In [309]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)


In [310]:
# Combine all tools into a list
tools = [wiki_tool, retriever_tool, arxiv_tool]

### 4. Create the Agent 

In [311]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

In [312]:
prompt = hub.pull("hwchase17/react-chat")

In [313]:
# We now use the standard `create_react_agent` from `langchain.agents`
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


### 5. Run the Agent

In [318]:
message = "Tell me about the NIT srinagar And what is its NIRF ranking  How is the college?"

In [319]:
output = agent_executor.invoke({
        "input":message ,
        "chat_history": []
    })
print(output['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: NIT srinagar NIRF ranking and college reviewDirector

 Prof. Binod Kumar Kanaujia





Welcome to NIT Srinagar, an institute of national importance serving the region for the past 6 decades. At NIT Srinagar, we firmly believe in nurturing a holistic learning environment that enables students to not only excel academically but also develop...




Read more


























About NIT Srinagar













                                                National Institute of Technology, Srinagar is one of the premier Educational Institutes in the Northern Regions of the country. 
                                                            It was established in 1960 and has been one of the eighteen Regional Engineering Colleges sponsored by the Govt. of India during the 2nd Plan.

The Institute is situated at the the banks of world-famous Dal Lake, with the far-fame

In [320]:
message = "Tell me something about machine learning"

output = agent_executor.invoke({
        "input":message ,
        "chat_history": []
    })
print(output['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Machine learning is a subfield of artificial intelligence (AI) that focuses on enabling systems to learn from data, identify patterns, and make decisions with minimal human intervention. Instead of being explicitly programmed for every task, machine learning algorithms are trained on large datasets, allowing them to improve their performance over time as they are exposed to more data. Key concepts include supervised learning (where the model learns from labeled data), unsupervised learning (where it finds patterns in unlabeled data), and reinforcement learning (where an agent learns through trial and error by interacting with an environment). It's used in a vast array of applications, from recommendation systems and natural language processing to medical diagnosis and autonomous vehicles.

> Finished chain.
Machine learning is a subfield of artificial intelligence (AI) that focuses on enabling sy

In [321]:
message = "tell me what's the paper is all about https://arxiv.org/abs/2509.09127"

output = agent_executor.invoke({
        "input":message ,
        "chat_history": []
    })
print(output['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: arxiv
Action Input: 2509.09127Published: 2025-09-11
Title: Anti-Money Laundering Machine Learning Pipelines; A Technical Analysis on Identifying High-risk Bank Clients with Supervised Learning
Authors: Khashayar Namdar, Pin-Chien Wang, Tushar Raju, Steven Zheng, Fiona Li, Safwat Do I need to use a tool? No
Final Answer: The paper titled "Anti-Money Laundering Machine Learning Pipelines; A Technical Analysis on Identifying High-risk Bank Clients with Supervised Learning" is about using machine learning, specifically supervised learning, to identify high-risk bank clients in the context of anti-money laundering efforts. It was published on September 11, 2025, and authored by Khashayar Namdar, Pin-Chien Wang, Tushar Raju, Steven Zheng, Fiona Li, and Safwat.

> Finished chain.
The paper titled "Anti-Money Laundering Machine Learning Pipelines; A Technical Analysis on Identifying High-risk Bank Clients wit

In [322]:
message = "tell me what's the paper is all about https://arxiv.org/abs/2509.09127 in detail"

output = agent_executor.invoke({
        "input":message ,
        "chat_history": []
    })
print(output['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: arxiv
Action Input: 2509.09127Published: 2025-09-11
Title: Anti-Money Laundering Machine Learning Pipelines; A Technical Analysis on Identifying High-risk Bank Clients with Supervised Learning
Authors: Khashayar Namdar, Pin-Chien Wang, Tushar Raju, Steven Zheng, Fiona Li, Safwat Do I need to use a tool? No
Final Answer: The paper titled "Anti-Money Laundering Machine Learning Pipelines; A Technical Analysis on Identifying High-risk Bank Clients with Supervised Learning" by Khashayar Namdar, Pin-Chien Wang, Tushar Raju, Steven Zheng, Fiona Li, and Safwat was published on September 11, 2025. Based on the title, it appears to be about using machine learning, specifically supervised learning, to identify high-risk bank clients for anti-money laundering purposes. However, I do not have access to the abstract or the full content of the paper to provide a more detailed summary.

> Finished chain.
The paper t